In [1]:
## Loading Libraries
import pandas as pd
import numpy as np
import sys
import jdatetime as jd
import os
import pickle
import io

# import power_index_calculator as px
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from ReadingBalancesheetData import read_blnc_data
from ReadingIncomeData import read_inc_data
from ReadingCashFlowData import read_cf_data
from convert_ar_characters import convert_ar_characters

# Creating شرکت-نماد dict from Balance Sheet Data

In [3]:
## Updating Sh_spec_handy based on the شرکت-نماد for years 1397-8 from balance sheet data
# Loading Sh_spec_handy data
os.chdir(r"C:\Users\Mahdi\Dr. Heidari - TSE\WorkInHands\_Data\OtherData")
Sh_spec_handy=pd.read_excel("Sh_spec_handy.xlsx")
Sh_spec_handy.Shareholder_raw=Sh_spec_handy.Shareholder_raw.apply(lambda x: convert_ar_characters(x))
Sh_spec_handy.Shareholder=Sh_spec_handy.Shareholder.apply(lambda x: convert_ar_characters(x))

os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
sherkat_namad = pd.DataFrame()
for year in range(1389,1399):
    file_name = 'BS'+str(year)[2:]+'.csv'
    df = pd.read_csv(file_name,header=1)
    df['شرکت']=df['شرکت'].apply(lambda x: convert_ar_characters(x))
    df['نماد']=df['نماد'].apply(lambda x: convert_ar_characters(x))
    sherkat_namad = sherkat_namad.append(df[['شرکت','نماد']]).drop_duplicates()
    
# Appending the data of two years
sherkat_namad = sherkat_namad.append(pd.DataFrame(data={'شرکت':['سر. صنعت و معدن'],'نماد':['وصنعت']}))
sherkat_namad = sherkat_namad.append(pd.DataFrame(data={'شرکت':['مهندسی فیروزا'],'نماد':['تفیرو']}))
sherkat_namad = sherkat_namad.append(pd.DataFrame(data={'شرکت':['تامین مواد اولیه فولاد صبانور'],'نماد':['کنور']}))
sherkat_namad = sherkat_namad.append(pd.DataFrame(data={'شرکت':['پتروشیمی فجر'],'نماد':['بفجر']}))

sherkat_namad.rename(columns={'شرکت':'Shareholder_raw','نماد':'Shareholder'},inplace=True)
sherkat_namad = sherkat_namad.drop_duplicates()

# Updating the Sh_spec_handy file
Sh_spec_handy = Sh_spec_handy.append(sherkat_namad).drop_duplicates()
Sh_spec_handy = Sh_spec_handy[Sh_spec_handy.Shareholder !='nan']
sherkat_namad_dict = dict(zip(Sh_spec_handy['Shareholder_raw'],Sh_spec_handy['Shareholder']))

# **Cleaning BalanceSheet Data**

In [7]:
df = pd.DataFrame()
for year in range(1389,1399):
    temp = read_blnc_data(year=year,path=r"C:\Users\Mahdi\OneDrive\Master Thesis\Data",sherkat_namad_dict=sherkat_namad_dict)
    df = df.append(temp['data'])
    
blnc_dict = pd.DataFrame(data={'نام اصلی':list(temp['columns_dict'].keys()),'نام استفاده شده':list(temp['columns_dict'].values())})    
blnc_dict.to_excel('Blnc_columns_dict.xlsx')

## Loading Balance sheet DATA
for year in range(1389,1399):
    out_name = "BalanceSheet"+str(year)+".csv"
    df_blnc = df[df.Year==year]
    os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
    df_blnc.to_csv(out_name)

In [8]:
blnc_dict
df_blnc.head()

,نام اصلی,نام استفاده شده
0,نماد,Symbol
1,شرکت,Firm
2,سال مالی,Fin_year
3,جمع دارایی‌های جاری,Tot_current_asset
4,تاریخ مصوب,Approve_date
5,خالص دارایی‌های ثابت,PPE
6,سرمایه گذاری‌ها و سایر دارایی‌ها,other_longterm_asset
7,جمع بدهی‌های جاری,Tot_current_lib
8,جمع کل دارایی‌ها,Tot_asset
9,جمع دارایی‌های غیر جاری,Tot_noncurrent_asset


,Firm,Symbol,Fin_year,Year,Approve_date,Tot_current_asset,PPE,other_longterm_asset,Tot_noncurrent_asset,Tot_asset,...,Tot_lib,Capital,Acummulated_profit,Saving_required,Saving_other,Equity,Debt_and_equity,Book_value,debt_equity_normal,public_rights
39,اقتصادی نگین گردشگری ایرانیان,گنگین,1398-09-30,1398,1398-12-25,980478.0,4778.0,12928956.0,12933846.0,13914324.0,...,7466542.0,3000000.0,3147782.0,300000.0,NaN,6447782.0,13914324.0,6447782.0,3957782.0,NaN
46,انرژی آذرآب,فنرژی,1398-09-30,1398,1398-12-25,873842.0,4864.0,4705.0,38180.0,912022.0,...,2337303.0,163160.0,-1597634.0,4743.0,4450.0,-1425281.0,912022.0,-1425281.0,NaN,NaN
57,ایران دارو,دیران,1398-09-30,1398,1399-01-31,1752398.0,138440.0,527.0,161236.0,1913634.0,...,1126786.0,530000.0,221821.0,34988.0,39.0,786848.0,1913634.0,786848.0,580148.0,NaN
94,بسته بندی ایران,فبیرا,1398-09-30,1398,1398-12-15,726298.0,1481420.0,1924.0,1484149.0,2210447.0,...,978491.0,1772732.0,-545767.0,4523.0,NaN,1231956.0,2210447.0,1231956.0,1231488.0,NaN
96,بسته بندی مشهد,فبستم,1398-09-30,1398,1398-12-01,20883.0,8948.0,7920.0,16983.0,37866.0,...,55214.0,51891.0,-70239.0,1000.0,NaN,-17348.0,37866.0,-17348.0,NaN,NaN


# **Cleaning Income Statement Data**

In [9]:
# Reading all data and appending
all_inc = pd.DataFrame()
for year in range(1390,1399):
    temp = read_inc_data(year,path=r"C:\Users\Mahdi\OneDrive\Master Thesis\Data",sherkat_namad_dict=sherkat_namad_dict)
    all_inc = all_inc.append(temp['data'])

# Creating dict for persian-english column names
Inc_dict = pd.DataFrame(data={'نام اصلی':list(temp['columns_dict'].keys()),'نام استفاده شده':list(temp['columns_dict'].values())})
Inc_dict.to_excel('Inc_columns_dict.xlsx')

# Saving Data
for year in list(all_inc.Year.drop_duplicates()):
    out_name = "IncomeStatement"+str(year)+".csv"
    df_inc = all_inc[all_inc.Year==year]
    os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
    df_inc.to_csv(out_name)

In [10]:
Inc_dict
df_inc.head()

,نام اصلی,نام استفاده شده
0,نام شرکت,Firm
1,نماد,Symbol
2,سال مالی,Fin_year
3,تاریخ مصوب,Approve_date
4,ارزش روز,Marketcap
5,سرمایه,Capital
6,جمع درآمدها,Revenue
7,بهای تمام شده کالای فروش رفته,Cost_of_revenue
8,هزینه تامین منابع مالی عملیات لیزینگ (هزینه سو...,capital_cost_
9,سود ناویژه,Gross_profit


,Firm,Symbol,Fin_year,Year,Approve_date,Revenue,Cost_of_revenue,Gross_profit,Operating_expenses,Operating_income,...,divideable_profit,exchange_rate_profit,ps,profit_to_sale,eps_,minority_share_from_profit_,capital_cost_,real_estate_income_,green_profit_,adj_investments_
8,سر. غدیر,وغدیر,1398-09-30,1398,1399-02-07,40839449.0,NaN,40839449.0,281464.0,40450003.0,...,50292455.0,NaN,19.39,95.6696,543.0,NaN,NaN,NaN,NaN,NaN
17,سر. امید,وامید,1398-10-30,1398,1399-01-31,34958155.0,NaN,34958155.0,153943.0,34905373.0,...,NaN,NaN,14.80,92.2323,1075.0,NaN,NaN,NaN,NaN,NaN
24,سنگ آهن گهر زمین,کگهر,1398-09-30,1398,1399-02-02,53125195.0,24562171.0,28563024.0,144189.0,25955252.0,...,NaN,NaN,8.06,48.814,2108.0,NaN,NaN,NaN,NaN,NaN
44,صنایع شیمیایی ایران,شیران,1398-09-30,1398,1398-12-04,18430522.0,13008511.0,5422011.0,969812.0,5749410.0,...,10950815.0,NaN,12.07,57.6555,1317.0,NaN,NaN,NaN,NaN,NaN
74,پرداخت الکترونیک سامان کیش,سپ,1398-09-30,1398,1398-12-24,16446448.0,13614112.0,2832336.0,NaN,2610480.0,...,NaN,NaN,7.86,16.602,910.0,NaN,NaN,NaN,NaN,NaN


# **Cleaning Cash Flow Statement Data**

In [11]:
path = r"C:\Users\Mahdi\OneDrive\Master Thesis\Data"

df_short = pd.DataFrame()
df_all = pd.DataFrame()

for year in range(1389,1399):
    temp = read_cf_data(year=year,path=path,sherkat_namad_dict=sherkat_namad_dict)
    df_short = df_short.append(temp['data_short'])
    df_all = df_all.append(temp['data_all'])    
    
CF_dict = pd.DataFrame(data={'نام اصلی':list(temp['columns_dict'].keys()),'نام استفاده شده':list(temp['columns_dict'].values())})
CF_dict.to_excel('CF_columns_dict.xlsx')    


for year in range(1389,1399):
    out_name_short = 'CashFlow'+str(year)+'.csv'
    out_name_all = 'CashFlow'+str(year)+'_all.csv'
    temp_short = df_short[df_short.Year==year]
    temp_all = df_all[df_all.Year==year]
    os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
    temp_short.to_csv(out_name_short)
    temp_all.to_csv(out_name_all)

In [12]:
temp_short.head()

,Year,Firm,Symbol,Fin_year,Operating_cash_flow,Investing_cash_flow,Financing_cash_flow,Begining_cash_position,Change_in_cash,End_cash_position
625,1398,گوشت مرغ ماهان,زماهان,1398-09-30,-12532.0,-50848.0,74166.0,51004.0,-20299.0,30705.0
15,1398,آسیا سیر ارس,حآسا,1398-12-29,130687.0,-14555.0,-36720.0,6854.0,55955.0,62809.0
20,1398,آهن و فولاد ارفع,ارفع,1398-12-29,5219222.0,-725253.0,1179407.0,4771820.0,-91699.0,4680121.0
27,1398,ارتباطات سیار,همراه,1398-12-29,88315574.0,-67357189.0,16012412.0,1593416.0,1464373.0,3057789.0
37,1398,اقتصادی نگین گردشگری ایرانیان,گنگین,1398-09-30,2124276.0,-42.0,-1240000.0,41761.0,-36129.0,5632.0


# **Cleaning Dividend Data**

In [13]:
os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
dividend_data = pd.read_excel('Annual_div-98.xlsx')
dividend_data = dividend_data.sort_values(by='year')
dividend_data = dividend_data[dividend_data.year>=1389]
dividend_data['firm'] = dividend_data['firm'].apply(lambda x: convert_ar_characters(x))
dividend_data['symbol'] = dividend_data['firm'].map(sherkat_namad_dict,na_action='ignore')
dividend_data['industry'] = dividend_data['industry'].apply(lambda x: convert_ar_characters(x))
dividend_data.rename(columns={'capital':'Equity','symbol':'Symbol','firm':'Firm','year':'Year','industry':'Industry','ind_id':'Ind_id',
                             'earning':'Earning','dividend':'Dividend','payout':'Payout','eps':'EPS','dps':'DPS'},inplace=True)

dividend_data = dividend_data[['Year', 'Firm', 'Symbol', 'Industry', 'Earning','Dividend','Payout','EPS', 'DPS', 'Equity',
                               'adj_eps', 'adj_dps', 'adj_earning', 'adj_dividend', 'cpi','Ind_id']]

In [14]:
# Unit converter
for x in ['Earning','Dividend','adj_earning', 'adj_dividend']:
    dividend_data[x] = dividend_data[x]*1000

dividend_data['Payout'] = np.round(dividend_data['Dividend']/dividend_data['Earning'],4)

In [15]:
for year in list(dividend_data.Year.drop_duplicates()):
    out_name = 'Dividend'+str(year)+'.csv'
    df = dividend_data[dividend_data.Year == year]
    os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
    df.to_csv(out_name)

In [16]:
df.head()

,Year,Firm,Symbol,Industry,Earning,Dividend,Payout,EPS,DPS,Equity,adj_eps,adj_dps,adj_earning,adj_dividend,cpi,Ind_id
2360,1397,ایرکا پارت صنعت,خکار,خودرو و ساخت قطعات,4250.000000,1000.0,0.2353,85,20,562808.0,NaN,NaN,NaN,NaN,NaN,34
7529,1397,فولاد کاوه جنوب کیش,کاوه,فلزات اساسی,944900.000000,990000.0,1.0477,859,900,17018133.0,NaN,NaN,NaN,NaN,NaN,27
7051,1397,البرزدارو,دالبر,مواد و محصولات دارویی,151218.000000,133920.0,0.8856,813,720,3920279.0,NaN,NaN,NaN,NaN,NaN,43
1884,1397,مهرکام پارس,خمهر,خودرو و ساخت قطعات,-43379.228791,0.0,-0.0000,-233,0,1014661.0,NaN,NaN,NaN,NaN,NaN,34
4603,1397,بیمه ملت,ملت,بیمه وصندوق بازنشستگی به جزتامین اجتماعی,86070.000000,71250.0,0.8278,302,250,4507619.0,NaN,NaN,NaN,NaN,NaN,66
